In [1]:
import pandas as pd
import numpy as np

In [7]:
# Load datasets
train = pd.read_csv("./../datasets/train.csv")
features = pd.read_csv("./../datasets/features.csv")
stores = pd.read_csv("./../datasets/stores.csv")

# Convert Date to datetime
train["Date"] = pd.to_datetime(train["Date"])
features["Date"] = pd.to_datetime(features["Date"])
print("train date:\n", train.head())
print("features date:\n", features.head())
print("stores date:\n", stores.head())

train date:
    Store  Dept       Date  Weekly_Sales  IsHoliday
0      1     1 2010-02-05      24924.50      False
1      1     1 2010-02-12      46039.49       True
2      1     1 2010-02-19      41595.55      False
3      1     1 2010-02-26      19403.54      False
4      1     1 2010-03-05      21827.90      False
features date:
    Store       Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  \
0      1 2010-02-05        42.31       2.572        NaN        NaN        NaN   
1      1 2010-02-12        38.51       2.548        NaN        NaN        NaN   
2      1 2010-02-19        39.93       2.514        NaN        NaN        NaN   
3      1 2010-02-26        46.63       2.561        NaN        NaN        NaN   
4      1 2010-03-05        46.50       2.625        NaN        NaN        NaN   

   MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN  211.242170         8.106   

In [4]:
# Merge train + features
df = train.merge(features, on=["Store", "Date"], how="left")

# Merge with stores metadata
df = df.merge(stores, on="Store", how="left")

print("Final dataset shape:", df.shape)
print(df.head())

Final dataset shape: (421570, 17)
   Store  Dept       Date  Weekly_Sales  IsHoliday_x  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50        False        42.31       2.572   
1      1     1 2010-02-12      46039.49         True        38.51       2.548   
2      1     1 2010-02-19      41595.55        False        39.93       2.514   
3      1     1 2010-02-26      19403.54        False        46.63       2.561   
4      1     1 2010-03-05      21827.90        False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
0        NaN        NaN        NaN        NaN        NaN  211.096358   
1        NaN        NaN        NaN        NaN        NaN  211.242170   
2        NaN        NaN        NaN        NaN        NaN  211.289143   
3        NaN        NaN        NaN        NaN        NaN  211.319643   
4        NaN        NaN        NaN        NaN        NaN  211.350143   

   Unemployment  IsHoliday_y Type    Size  
0 

In [8]:
# Check missing values
print(df.isnull().sum())

# Fill markdowns with 0 (no promotion that week)
for col in ["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]:
    df[col] = df[col].fillna(0)

# Fill CPI/Unemployment with forward fill (carry last value)
df["CPI"] = df["CPI"].fillna(method="ffill")
df["Unemployment"] = df["Unemployment"].fillna(method="ffill")

Store                0
Dept                 0
Date                 0
Weekly_Sales         0
IsHoliday_x          0
Temperature          0
Fuel_Price           0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday_y          0
Type                 0
Size                 0
dtype: int64


C:\Users\User\AppData\Local\Temp\ipykernel_17820\3276043646.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["CPI"] = df["CPI"].fillna(method="ffill")
C:\Users\User\AppData\Local\Temp\ipykernel_17820\3276043646.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Unemployment"] = df["Unemployment"].fillna(method="ffill")


In [10]:
# Date features
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week.astype(int)

# Lag features (previous weeks’ sales)
df = df.sort_values(["Store","Dept","Date"])
df["Lag1"] = df.groupby(["Store","Dept"])["Weekly_Sales"].shift(1)
df["Lag4"] = df.groupby(["Store","Dept"])["Weekly_Sales"].shift(4)

# Fill NA lags with 0
df[["Lag1","Lag4"]] = df[["Lag1","Lag4"]].fillna(0)
print(df.head())

   Store  Dept       Date  Weekly_Sales  IsHoliday_x  Temperature  Fuel_Price  \
0      1     1 2010-02-05      24924.50        False        42.31       2.572   
1      1     1 2010-02-12      46039.49         True        38.51       2.548   
2      1     1 2010-02-19      41595.55        False        39.93       2.514   
3      1     1 2010-02-26      19403.54        False        46.63       2.561   
4      1     1 2010-03-05      21827.90        False        46.50       2.625   

   MarkDown1  MarkDown2  MarkDown3  ...         CPI  Unemployment  \
0        0.0        0.0        0.0  ...  211.096358         8.106   
1        0.0        0.0        0.0  ...  211.242170         8.106   
2        0.0        0.0        0.0  ...  211.289143         8.106   
3        0.0        0.0        0.0  ...  211.319643         8.106   
4        0.0        0.0        0.0  ...  211.350143         8.106   

   IsHoliday_y  Type    Size  Year  Month  Week      Lag1     Lag4  
0        False     A  151315 

In [11]:
# Drop rows with missing target
df = df.dropna(subset=["Weekly_Sales"])

# Save clean dataset
df.to_csv("./../datasets/clean_walmart.csv", index=False)

print("✅ Preprocessing done! Clean dataset saved.")

✅ Preprocessing done! Clean dataset saved.
